In [1]:
!pip install gradio pymupdf torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
import gradio as gr
from transformers import pipeline
import fitz

In [3]:
# Initialize summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to summarize text
def summarize_text(text, model, max_length=1024):
    # Split the input text into smaller chunks
    chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]

    # Summarize each chunk separately (You can use list comprehension)
    summaries = []
    for chunk in chunks:
        summary = model(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
        summaries.append(summary)

    # Concatenate the summaries into a single string
    summary = ' '.join(summaries)

    return summary


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
# Function to read PDF and summarize
def summarize_pdf(pdf_file, model):
    with fitz.open(pdf_file.name) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return summarize_text(text, model)


In [5]:
def summarize(input_type, input_text, uploaded_file):
    try:
        if input_type == "Text":
            summary = summarize_text(input_text, summarizer)
        else:
            summary = summarize_pdf(uploaded_file, summarizer)
        return summary
    except Exception as e:
        return "There was a problem summarizing the text. Please try again later."

In [7]:
# Define the footer
footer = """
<div style="text-align: center; margin-top: 20px;">
    <a href="https://www.linkedin.com/in/arken-bourezak-016744209/" target="_blank">LinkedIn</a>

    <br>
    Made with 💖 by ARKEN BOUREZAK
</div>
"""

In [8]:
# Define the inputs and outputs
inputs = [
    gr.Radio(["Text", "PDF"], label="Input Type"),
    gr.Textbox(lines=10, label="Enter Text to Summarize", visible=False),
    gr.File(label="Upload PDF file", visible=False)
]
outputs = [
    gr.Textbox(label="Summary"),
    gr.HTML(footer)
]


In [9]:
# Define the submit button
submit_btn = gr.Button("Submit")

In [10]:
# Define the Gradio interface
with gr.Blocks(theme='gradio/soft') as app:
    gr.Markdown("# Text and PDF Summarization App")
    gr.Markdown("Note: This model can handle a maximum of 1024 tokens. A token is a unit of text that the model can process at a time. When summarizing text, the input text is split into smaller chunks of up to 1024 tokens each, and each chunk is summarized separately. The summaries are then concatenated into a single summary.")
    with gr.Row():
        input_type = gr.Radio(["Text", "PDF"], label="Input Type")
    with gr.Row():
        input_text = gr.Textbox(lines=10, label="Enter Text to Summarize", visible=False)
        uploaded_file = gr.File(label="Upload PDF file", visible=False)
    with gr.Row():
        submit_btn = gr.Button("Submit")
    with gr.Row():
        summary = gr.Textbox(label="Summary")
    with gr.Row():
        footer = gr.HTML(footer)
    # Define the change event handler for the input type radio buttons
    def input_type_change(input_type):
        if input_type == "Text":
            return {input_text: gr.Textbox(visible=True), uploaded_file: gr.File(visible=False)}
        else:
            return {input_text: gr.Textbox(visible=False), uploaded_file: gr.File(visible=True)}
    input_type.change(fn=input_type_change, inputs=[input_type], outputs=[input_text, uploaded_file])

    # Define the click event handler for the submit button
    submit_btn.click(fn=summarize, inputs=[input_type, input_text, uploaded_file], outputs=[summary])
# Launch the Gradio interface
if __name__ == "__main__":
    app.launch()


themes/theme_schema@0.0.3.json:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6b1a278743d75f88b9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
